## Battle of the Neighborhoods – IBM Capstone Project
## Where to develop a Karaoke Bar 

### Introduction
A small investment company inquires where they should best allocate their resources to develop a Karaoke-themed bar. They are looking for an area where starting a new business will attract the right customers and generate the highest revenue. 

### Background
They’ve been racking their brains trying to pinpoint a city, but the group just can’t come to a consensus. Luckily, they heard about you through a mutual colleague that you specialize in data analytics and could come up with best possible solutions based on data. What they’re looking for. Generally speaking, they’re looking to define customers spending patterns in various localities to determine the best U.S. based location to set up their karaoke business. 

### Business Problem
In order to help the investment company from maximizing profits and minimizing loss, I will definitely be incorporating data from websites to define which areas are the most densely populated, have higher than average incomes, use the Foursquare API to obtain venues in various U.S. cities, then categorize the venues based on proportional weights to designate value. Once the city(s) is determined, we can narrow in on distinct neighborhoods and obtain visual representation of the coordinates. In particular, we want to focus on the venues listed for each locality (using Foursquare API) and determine why a certain location could be considered more appealing.

### Data
Data will be scraped from (https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population) and (https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income) to obtain a pandas dataframe using the ‘BeautifulSoup’ library discussed in the modules. Clean the data and pre-process for exploratory data analysis. The two Wikipedia websites should provide enough information to determine which location would provide the best potential return on investment for the company.

# Let's start digging into the data and import the necessary libraries for further analysis

In [1]:

import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim #obtain geo-coordinates
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import xml
import folium # map rendering library
print('Libraries we will need')

Libraries we will need


## Let's start with the list of U.S. cities (by population) from Wikipedia by scraping the webpage 

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

In [3]:
table = soup.find_all('table')[4]

Extract table and create dataframe 

In [4]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head()

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 11 columns):
Rank                           314 non-null object
City                           314 non-null object
State                          314 non-null object
del1                           314 non-null object
del2                           314 non-null object
del3                           314 non-null object
Sq.Area                        314 non-null object
del5                           314 non-null object
population density in Sq Mi    314 non-null object
Population density in Km2      314 non-null object
Location                       314 non-null object
dtypes: object(11)
memory usage: 27.1+ KB


Determine city radius using Sq.Area 

In [6]:
new = df["Sq.Area"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area"] = new.str.replace(',','')
df["Sq.Area"] = df["Sq.Area"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area"])

Clean dataframe by dropping unneccessary columns

In [7]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density in Sq Mi"], inplace = True)
df.head(15)

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824
5,Philadelphia[e],Pennsylvania,"4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...,11.584472
6,San Antonio,Texas,"1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...,21.470911
7,San Diego,California,"1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...,18.033303
8,Dallas,Texas,"1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...,18.463477
9,San Jose,California,"2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...,13.322913


Displaying Geocoordinates for cities (since we will need it for later analysis using Foursquare API)

In [8]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head(15)

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824
5,Philadelphia[e],Pennsylvania,"4,511/km2",﻿40.0094°N 75.1333°W﻿,11.584472
6,San Antonio,Texas,"1,250/km2",﻿29.4724°N 98.5251°W﻿,21.470911
7,San Diego,California,"1,670/km2",﻿32.8153°N 117.1350°W﻿,18.033303
8,Dallas,Texas,"1,493/km2",﻿32.7933°N 96.7665°W﻿,18.463477
9,San Jose,California,"2,231/km2",﻿37.2967°N 121.8189°W﻿,13.322913


In [9]:
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)

In [10]:
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
k.head()
df = k.copy(deep = True)

In [11]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head(8)

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901
5,Philadelphia[e],Pennsylvania,"4,511/km2",11584.472366,40.0094,-75.1333
6,San Antonio,Texas,"1,250/km2",21470.910554,29.4724,-98.5251
7,San Diego,California,"1,670/km2",18033.302526,32.8153,-117.1350


## Lets look at the U.S. city plots using Folium and Info extracted from Wikipedia

In [12]:
import folium 
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=3)

# add markers to map
for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

Foursquare API

In [13]:
CLIENT_ID = '4NR5KPRBA2R1Q5PQ425J4HL2FH3WPF4ZC055RJVRC5LRTENW' # your Foursquare ID
CLIENT_SECRET = '4WPH5YCXNPJNFVKNOJOP5BAVBHEUGZPJJIKIKCBOVOFROH1D' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: 4NR5KPRBA2R1Q5PQ425J4HL2FH3WPF4ZC055RJVRC5LRTENW
CLIENT_SECRET:4WPH5YCXNPJNFVKNOJOP5BAVBHEUGZPJJIKIKCBOVOFROH1D


Extract Columns Into dataframe using json and API

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
location_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

New York[d]


KeyError: 0

In [ ]:
location_venues

## Assign weights to foursquare categories to discern which are best suited for the Karaoke establishment

In [ ]:
k = location_venues.copy(deep = True)
weights_dict={'Movie Theater':3,'Beach':3,'Concert Hall':2.5,'Playground':3,'Coffee Shop':3.5,'Food Court':4,'Nightclub':4,'Toy / Game Store':4.5,'Theme Park Ride / Attraction':4,'Pub':4}
data = location_venues['Venue Category']
allVenues = data.astype(list)

In [ ]:
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
location_venues['weights'] = weights;
location_venues.head()

## Drop NaN rows for weights

In [ ]:
location_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
location_venues.head()

Group venues by city and calculate mean values

In [ ]:
citywise_venues_weights = location_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

Merge table for calculated weights (mean)

In [ ]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density in Km2','weights']].copy()
city_selection.head()

In [ ]:
k = city_selection.copy(deep = True)
k['Population density in Km2'] = k['Population density in Km2'].str.split("/", n = 0, expand = True)
k['Population density in Km2'] = k['Population density in Km2'].str.replace(',','')
k['Population density in Km2'] = k['Population density in Km2'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head()

NORMALIZE

In [ ]:
from sklearn import preprocessing
column_names_to_normalize = ['Population density in Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

Calculate sum of normalized columns (determining most suitable locality)

In [ ]:
city_selection['sum'] = city_selection['Population density in Km2'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name

In [ ]:
#which state
row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

## Let's check income levels of New Jersey to be sure it fits the additional requirement

In [ ]:
p_row = df_state.loc[df_state['State'] == state_name].index[0]
per_capital_income = df_state['Per capita income'].iloc[p_row]
print("Per capita income of New Jercy is :", per_capital_income)

> $50000 suggests New Jersey has strong potential

Let's dive deeper and get venues near that location

In [ ]:
lat_newJercy = df['Latitude'].iloc[row]
long_newJercy = df['Longitude'].iloc[row]
print(lat_newJercy, long_newJercy)

In [ ]:
def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


new_jersey_venues = getNearbyVenues1(name = 'Jersey City', latitudes = lat_newJercy ,longitudes = long_newJercy, radius = 2500)
new_jersey_venues.head()

In [ ]:
venues_in_newjersey = new_jersey_venues.copy(deep = True)
venues_in_newjersey.shape

In [ ]:
venues_in_newjersey.describe()

Let's take a look at the weights for all categories 

In [ ]:
k = new_jersey_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':4,
'Restaurant':5,
'State / Provincial Park':1,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':3,
'Park':2,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':3,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':1,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':1,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':2,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':4,
'Burger Joint':5,
'Cheese Shop':1,
'Coffee Shop':1,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':2,
'Gym':2,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Café':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'American Restaurant':4,
'American Restaurant':5,
'Café':1,
'New American Restaurant':1,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':2,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':1,
'Brewery':1,
'Burger Joint':3,
'Burger Joint':2,
'Café':2,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':3,
'Vietnamese Restaurant':2,
'Italian Restaurant':3,
'Pet Store':3}

Time to Plot using Python Visuals

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_newJercy, long_newJercy], zoom_start=15) # generate map centred around Jersey city


# add Jersey City as a red circle mark
folium.features.CircleMarker(
    [lat_newJercy, long_newJercy],
    radius=10,
    popup='Jersey city',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [ ]:
for lat, lng, label in zip(venues_in_newjersey['Venue Latitude'], venues_in_newjersey['Venue Longitude'], venues_in_newjersey['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

Assign Weights

In [ ]:
allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

In [ ]:
newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe

K-Means Cluster using calculated weights

In [ ]:
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

In [ ]:
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

In [ ]:
lat1 = 40.720102
long1 = -74.04812

In [ ]:
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Gaming arcade can be installed within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map